# <center>Applying SNA on Humanities Data</center>

# Introduction

In this section, we will apply the skills we have learned about SNA on real humanities data. Again, we will be working with the Volume 7 Dataset from the South African TRC. Unlike the last chapter on Topic Modeling, we will not be interested in how the descriptions of violence cluster together. Instead, we will be interested in exploring victim relationships to specific organizations in their description. It is important to note here, that we do not in this approach know the relationship between the victim and the organization. There is equal possibility that they were a member or victim of the organization.

## Examining the Data

In [2]:
import pandas as pd

In [54]:
df = pd.read_csv("../data/trc.csv")
# df = df[["Last", "First", "Description","ORG", "Place"]]

In [55]:
df

,ObjectId,Last,First,Description,Place,Yr,Homeland,Province,Long,Lat,HRV,ORG
0,1,AARON,Thabo Simon,An ANCYL member who was shot and severely inju...,Bethulie,1991.0,NaN,Orange Free State,25.97552,-30.503290,shoot|injure,ANC|ANCYL|Police|SAP
1,2,ABBOTT,Montaigne,A member of the SADF who was severely injured ...,Messina,1987.0,NaN,Transvaal,30.039597,-22.351308,injure,SADF
2,3,ABRAHAM,Nzaliseko Christopher,A COSAS supporter who was kicked and beaten wi...,Mdantsane,1985.0,Ciskei,Cape of Good Hope,27.6708791,-32.958623,beat,COSAS|Police
3,4,ABRAHAMS,Achmat Fardiel,Was shot and blinded in one eye by members of ...,Athlone,1985.0,NaN,Cape of Good Hope,18.50214,-33.967220,shoot|blind,SAP
4,5,ABRAHAMS,Annalene Mildred,Was shot and injured by members of the SAP in ...,Robertson,1990.0,NaN,Cape of Good Hope,19.883611,-33.802220,shoot|injure,Police|SAP
...,...,...,...,...,...,...,...,...,...,...,...,...
20829,20888,XUZA,Mandla,Was severely injured when he was stoned by a f...,Carletonville,1991.0,NaN,Transvaal,27.397673,-26.360943,injure|stone,ANC
20830,20889,YAKA,Mbangomuni,An IFP supporter and acting induna who was sho...,Mvutshini,1993.0,KwaZulu,Natal,30.28172,-30.868900,shoot,NaN
20831,20890,YALI,Khayalethu,"Was shot by members of the SAP in Lingelihle, ...",Cradock,1986.0,NaN,Cape of Good Hope,25.619176,-32.164221,shoot,SAP
20832,20891,YALO,Bikiwe,An IFP supporter whose house and possessions w...,Port Shepstone,1994.0,NaN,Natal,30.4297304,-30.752126,destroy,ANC


In [8]:
from pyvis.network import Network
import networkx as nx

In [56]:
nodes = []
edge_list = []
df = df[:1000]
for idx, row in df.iterrows():
    node_id = f"{idx}_{row.First} {row.Last}"
    place = None
    if pd.isnull(row.Place) == False:
        place = row.Place
    homeland = None
    if pd.isnull(row.Homeland) == False:
        homeland = row.Homeland
    nodes.append((node_id, {"color": "green", "place": place}))
    if pd.isnull(row.ORG) == False:
        orgs = row.ORG.split("|")
        for org in orgs:
            if (org, {"color": "blue"}) not in nodes:
                nodes.append((org, {"color": "blue"}))
            edge_list.append((node_id, org, {"place": place, "homeland": homeland}))
print(nodes[:1])
print(edge_list[:1])

[('0_Thabo Simon AARON', {'color': 'green', 'place': 'Bethulie'})]
[('0_Thabo Simon AARON', 'ANC', {'place': 'Bethulie', 'homeland': None})]


In [57]:
G = nx.Graph()
G.add_nodes_from(nodes)
G.add_edges_from(edge_list)

In [58]:
net = Network(notebook=True, filter_menu=True, select_menu=True)
net.from_nx(G)
net.toggle_physics(False)
net.show_buttons(filter_=['physics'])

Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 


In [59]:
net.save_graph("trc_graph.html")

In [19]:
net = Network(notebook=True, filter_menu=True, select_menu=True)
net.barnes_hut()
small_df = df[:100]
for idx, row in small_df.iterrows():
    node_id = f"{idx}_{row.First} {row.Last}"
    place = None
    if pd.isnull(row.Place) == False:
        place = row.Place
    net.add_node(node_id, place=place, title=node_id)
    if pd.isnull(row.ORG) == False:
        orgs = row.ORG.split("|")
        for org in orgs:
            if org not in [node.title for node in net.get_nodes()]:
                net.add_node(org, color="red", title=org)
            net.add_edge(node_id, org, place=place)

Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 


In [20]:
net.toggle_physics(False)
net.show_buttons(filter_=['physics'])
net.save_graph("trc_graph.html")

In [21]:
from IPython.display import IFrame
IFrame("trc_graph.html", width=1000, height=1000)

## Adding Colors to Nodes

## Adding Metadata to Nodes